# An OpenAPI planner with LMQL and Langchain

Directly inspired by the Langchain [tutorial](https://python.langchain.com/en/latest/modules/agents/toolkits/examples/openapi.html) on OpenAPI agents

In [1]:
import os
os.environ["OPENAI_API_KEY"] = # Add your OpenAI API key here

## Retrieval of the Spotify OpenAPI specification

In [2]:
!wget https://raw.githubusercontent.com/APIs-guru/openapi-directory/main/APIs/spotify.com/1.0.0/openapi.yaml
!mv openapi.yaml spotify_openapi.yaml

import yaml
from langchain.agents.agent_toolkits.openapi.spec import reduce_openapi_spec

with open("spotify_openapi.yaml") as f:
    raw_spotify_api_spec = yaml.load(f, Loader=yaml.Loader)
spotify_api_spec = reduce_openapi_spec(raw_spotify_api_spec)

--2023-04-25 00:37:31--  https://raw.githubusercontent.com/APIs-guru/openapi-directory/main/APIs/spotify.com/1.0.0/openapi.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8003::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 289324 (283K) [text/plain]
Saving to: ‘openapi.yaml’

openapi.yaml        100%[===================>] 282.54K  --.-KB/s    in 0.007s  

2023-04-25 00:37:31 (40.0 MB/s) - ‘openapi.yaml’ saved [289324/289324]



## Use of the OpenAI planner agent in Langchain

In [3]:
from langchain.llms.openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.agents.agent_toolkits.openapi.planner_prompt import API_PLANNER_PROMPT

llm = OpenAI(model_name="text-davinci-003", temperature=0.0)

endpoint_descriptions = [
    f"{name}\n{description}\n" for name, description, _ in spotify_api_spec.endpoints
]

endpoint_names = set(name for name, _, _ in spotify_api_spec.endpoints)

prompt = PromptTemplate(
    template=API_PLANNER_PROMPT,
    input_variables=["query"],
    partial_variables={"endpoints": "- " + "- ".join(endpoint_descriptions)},
)

chain = LLMChain(
    llm=llm,
    prompt=prompt
)

In [4]:
query = "make me a playlist with the first song from kind of blue. call it machine blues."
result = chain(query)
print(result["text"])


1. GET /search to search for the album Kind of Blue
2. GET /albums/{id} to get the album's id
3. GET /albums/{id}/tracks to get the tracks from the album
4. GET /me to get the user's id
5. POST /users/{id}/playlists to create a playlist with the name "Machine Blues"
6. POST /playlists/{playlist_id}/tracks to add the first track from the album to the playlist


As suggested in the Langchain tutorial, `text-davinci-003` struggles with this task. Here we see that an endpoint mentioned in the plan is invalid

In [5]:
import re

def get_invalid_endpoints(s):
    result = []
    pattern = r"\b(GET|POST|PATCH|DELETE)\s+(/\S+)*"
    matches = re.findall(pattern, s)
    for x, y in matches:
        endpoint = f"{x} {y}"
        if endpoint not in endpoint_names:
            result.append(endpoint)
    return result

get_invalid_endpoints(result["text"])

['POST /users/{id}/playlists']

## Creation of an OpenAI planner agent in LMQL, with additional constraints

In [6]:
import lmql
import nest_asyncio
nest_asyncio.apply()


@lmql.query
async def plan(query, endpoint_names):
    '''
    argmax(max_len=4096)
        "{prompt.format(query=query)} "
        for i in range(1, 8):
            "{i}. [ENDPOINT] [PURPOSE]"
            if PURPOSE[-1] != "\n":
                break
    from
        "openai/text-davinci-003"
    where
        ENDPOINT in set(endpoint_names) and
        STOPS_AT(PURPOSE, "\n")
    '''

With the LMQL agent, the plan doesn't include any invalid endpoint (it's however not correct because it doesn't get the user id with `GET /me` before creating the playlist...)

In [7]:
query = "make me a playlist with the first song from kind of blue. call it machine blues."
result_lmql = await plan(query, endpoint_names)

plan = result_lmql[0].prompt.split("Plan: ")[-1]
print(plan)
invalid = get_invalid_endpoints(plan)
if len(invalid) > 0:
    print(", ".join(invalid))

1. GET /search  with a query param to search for the album "Kind of Blue"
2. GET /albums/{id}/tracks  to get the track list for the album
3. GET /tracks/{id}  to get the track information for the first track
4. POST /users/{user_id}/playlists  to create a new playlist
5. POST /users/{user_id}/playlists  to add the track to the playlist
6. POST /playlists/{playlist_id}/tracks  to add the track to the playlist
7. POST /playlists/{playlist_id}/tracks  to rename the playlist to "Machine Blues"
